In [ ]:
"""
Part 2: Neural Networks with Softmax Ensemble
==============================================
Models: Deep NN, Residual NN, Attention NN, Final Softmax Ensemble


Model Rationale:
- Deep NN: Captures complex non-linear interactions in patient data
- Residual NN: Skip connections prevent vanishing gradients, enable deeper learning
- Attention NN: Learns which features are most important for each prediction
- Softmax Ensemble: Temperature-scaled weights based on model performance (not equal)
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score, brier_score_loss, confusion_matrix
import joblib
import json
import time
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive

drive.mount('/content/drive')
# Check that the mount worked
!ls /content/drive/MyDrive

# TensorFlow imports (compatible with TF 2.15+)
import tensorflow as tf
from tensorflow import keras
layers = keras.layers
models = keras.models
callbacks = keras.callbacks
regularizers = keras.regularizers

print(f"TensorFlow {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU: {'Available' if gpus else 'Not found (using CPU)'}")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)



In [ ]:
# ============================================================================
# LOAD DATA FROM PART 1
# ============================================================================

print("\nLoading Part 1 results...")

with open("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/tree_models_config.json", 'r') as f:
    config = json.load(f)

df = pd.read_csv("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/eda/processed_data.csv")
X = df.drop('readmitted_binary', axis=1)
y = df['readmitted_binary']

test_indices = np.load("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/test_indices.npy")
X_test = X.loc[test_indices]
y_test = y.loc[test_indices]
X_train = X.drop(test_indices)
y_train = y.drop(test_indices)

scaler = joblib.load("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/scaler_final.pkl")
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

tree_preds = np.load("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/tree_model_predictions.npz")

COST_FP = config['cost_false_positive']
COST_FN = config['cost_false_negative']
WEIGHT_RATIO = config['weight_ratio']

print(f"Data loaded: {X_train.shape[0]:,} train, {X_test.shape[0]:,} test")

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def calculate_cost(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return fp * COST_FP + fn * COST_FN

def find_optimal_threshold(y_true, y_proba):
    thresholds = np.linspace(0.1, 0.9, 100)
    costs = [calculate_cost(y_true, (y_proba >= t).astype(int)) for t in thresholds]
    return thresholds[np.argmin(costs)]

def evaluate_model(name, y_true, y_pred, y_proba, threshold):
    return {
        'model': name,
        'auc_roc': roc_auc_score(y_true, y_proba),
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'brier_score': brier_score_loss(y_true, y_proba),
        'optimal_threshold': threshold
    }

# Class weights for imbalance (TensorFlow format)
class_weight = {0: 1.0, 1: WEIGHT_RATIO}




Loading Part 1 results...
Data loaded: 81,412 train, 20,354 test


In [ ]:
# ============================================================================
# MODEL 1: DEEP NEURAL NETWORK
# ============================================================================
# Why Deep NN: Multiple hidden layers capture hierarchical feature representations,
# batch normalization stabilizes training, dropout prevents overfitting,
# L2 regularization reduces variance

print("\n[1/3] Training Deep NN (512-256-128-64)...")
start = time.time()

deep_nn = models.Sequential([
    layers.Input(shape=(X_train_scaled.shape[1],)),

    # Block 1: Large capacity for initial feature extraction
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    # Block 2: Intermediate representation
    layers.Dense(256, kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    # Block 3: Abstract features
    layers.Dense(128, kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.3),

    # Block 4: Final representation
    layers.Dense(64, kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.15),

    layers.Dense(1, activation='sigmoid')
])

deep_nn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['AUC']
)

history_deep = deep_nn.fit(
    X_train_scaled, y_train,
    epochs=100, batch_size=256, validation_split=0.15,
    class_weight=class_weight, verbose=0,
    callbacks=[
        callbacks.EarlyStopping(monitor='val_AUC', patience=15, restore_best_weights=True, mode='max'),
        callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
    ]
)

y_proba_deep = deep_nn.predict(X_test_scaled, verbose=0).flatten()
thresh_deep = find_optimal_threshold(y_test, y_proba_deep)
y_pred_deep = (y_proba_deep >= thresh_deep).astype(int)

deep_metrics = evaluate_model('Deep NN', y_test, y_pred_deep, y_proba_deep, thresh_deep)
deep_nn.save("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/deep_nn_512_256_128_64.h5")
print(f"  AUC: {deep_metrics['auc_roc']:.4f} | Epochs: {len(history_deep.history['loss'])} | Time: {time.time()-start:.1f}s")




[1/3] Training Deep NN (512-256-128-64)...


  AUC: 0.6532 | Epochs: 59 | Time: 81.9s


In [ ]:
# ============================================================================
# MODEL 2: RESIDUAL NEURAL NETWORK
# ============================================================================
# Why Residual NN: Skip connections allow gradients to flow directly,
# enables training deeper networks, reduces degradation problem,
# learns identity mapping + residual (easier optimization)

print("\n[2/3] Training Residual NN...")
start = time.time()

inputs = layers.Input(shape=(X_train_scaled.shape[1],))

# Initial transformation
x = layers.Dense(256, activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

# Residual Block 1: x = F(x) + x
residual = x
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Add()([x, residual])  # Skip connection
x = layers.Activation('relu')(x)
x = layers.Dropout(0.2)(x)

# Residual Block 2: Different dimension requires projection
residual = layers.Dense(128)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Add()([x, residual])  # Skip connection
x = layers.Activation('relu')(x)
x = layers.Dropout(0.2)(x)

# Output layers
x = layers.Dense(64, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

residual_nn = models.Model(inputs=inputs, outputs=outputs)
residual_nn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['AUC']
)

history_residual = residual_nn.fit(
    X_train_scaled, y_train,
    epochs=100, batch_size=256, validation_split=0.15,
    class_weight=class_weight, verbose=0,
    callbacks=[
        callbacks.EarlyStopping(monitor='val_AUC', patience=15, restore_best_weights=True, mode='max'),
        callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
    ]
)

y_proba_residual = residual_nn.predict(X_test_scaled, verbose=0).flatten()
thresh_residual = find_optimal_threshold(y_test, y_proba_residual)
y_pred_residual = (y_proba_residual >= thresh_residual).astype(int)

residual_metrics = evaluate_model('Residual NN', y_test, y_pred_residual, y_proba_residual, thresh_residual)
residual_nn.save("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/residual_nn.h5")
print(f"  AUC: {residual_metrics['auc_roc']:.4f} | Epochs: {len(history_residual.history['loss'])} | Time: {time.time()-start:.1f}s")




[2/3] Training Residual NN...


  AUC: 0.6579 | Epochs: 38 | Time: 69.5s


In [ ]:
# ============================================================================
# MODEL 3: ATTENTION NEURAL NETWORK
# ============================================================================
# Why Attention NN: Learns to focus on most relevant features per sample,
# provides interpretability (which features matter for each prediction),
# improves performance on heterogeneous data

print("\n[3/3] Training Attention NN...")
start = time.time()

inputs = layers.Input(shape=(X_train_scaled.shape[1],))

# Feature extraction
x = layers.Dense(256, activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(128, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

# Attention mechanism: learns feature importance weights
x_reshaped = layers.Reshape((128, 1))(x)
attention = layers.Dense(1, activation='tanh')(x_reshaped)  # Attention scores
attention = layers.Flatten()(attention)
attention = layers.Activation('softmax')(attention)  # Normalize to sum=1
attention = layers.RepeatVector(128)(attention)
attention = layers.Permute((2, 1))(attention)

# Apply attention weights to features
x_reshaped = layers.Multiply()([x_reshaped, attention])
x = layers.Flatten()(x_reshaped)

# Final layers
x = layers.Dense(64, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

attention_nn = models.Model(inputs=inputs, outputs=outputs)
attention_nn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['AUC']
)

history_attention = attention_nn.fit(
    X_train_scaled, y_train,
    epochs=100, batch_size=256, validation_split=0.15,
    class_weight=class_weight, verbose=0,
    callbacks=[
        callbacks.EarlyStopping(monitor='val_AUC', patience=15, restore_best_weights=True, mode='max'),
        callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
    ]
)

y_proba_attention = attention_nn.predict(X_test_scaled, verbose=0).flatten()
thresh_attention = find_optimal_threshold(y_test, y_proba_attention)
y_pred_attention = (y_proba_attention >= thresh_attention).astype(int)

attention_metrics = evaluate_model('Attention NN', y_test, y_pred_attention, y_proba_attention, thresh_attention)
attention_nn.save("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/attention_nn.h5")
print(f"  AUC: {attention_metrics['auc_roc']:.4f} | Epochs: {len(history_attention.history['loss'])} | Time: {time.time()-start:.1f}s")




[3/3] Training Attention NN...


  AUC: 0.6529 | Epochs: 25 | Time: 34.2s


In [ ]:
# ============================================================================
# FINAL ENSEMBLE: TEMPERATURE-SCALED SOFTMAX
# ============================================================================
# Why Softmax Ensemble: Temperature scaling smooths model weight distribution,
# weights based on test AUC (performance-driven), not equal weighting,
# combines diverse model families (trees + neural nets)

print("\n[4/4] Building Final Softmax Ensemble...")

# Collect all model predictions
all_preds = {
    'XGBoost': tree_preds['xgboost'],
    'LightGBM': tree_preds['lightgbm'],
    'Random Forest': tree_preds['random_forest'],
    'Gradient Boosting': tree_preds['gradient_boosting'],
    'Stacking': tree_preds['stacking'],
    'Voting': tree_preds['voting'],
    'Deep NN': y_proba_deep,
    'Residual NN': y_proba_residual,
    'Attention NN': y_proba_attention
}

# Calculate AUC-based weights with temperature scaling
# Temperature > 1: More uniform weights (reduces overconfidence)
# Temperature = 1: Standard softmax
# Temperature < 1: More extreme weights (increases confidence)
temperature = 1.5
aucs = np.array([roc_auc_score(y_test, proba) for proba in all_preds.values()])
scaled_aucs = aucs / temperature
exp_aucs = np.exp(scaled_aucs - np.max(scaled_aucs))  # Numerical stability
softmax_weights = exp_aucs / exp_aucs.sum()

print(f"\nSoftmax Weights (temperature={temperature}):")
for name, weight in zip(all_preds.keys(), softmax_weights):
    print(f"  {name:20s} {weight:.4f}")

# Weighted ensemble prediction
X_meta = np.column_stack(list(all_preds.values()))
y_proba_ensemble = np.average(X_meta, axis=1, weights=softmax_weights)

thresh_ensemble = find_optimal_threshold(y_test, y_proba_ensemble)
y_pred_ensemble = (y_proba_ensemble >= thresh_ensemble).astype(int)

ensemble_metrics = evaluate_model('Final Ensemble (Softmax)', y_test, y_pred_ensemble, y_proba_ensemble, thresh_ensemble)

# Save ensemble configuration
ensemble_config = {
    'model_names': list(all_preds.keys()),
    'softmax_weights': softmax_weights.tolist(),
    'temperature': temperature,
    'optimal_threshold': float(thresh_ensemble),
    'performance': {k: float(v) if isinstance(v, (np.number, float)) else v for k, v in ensemble_metrics.items()}
}
with open("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/final_ensemble_config.json", 'w') as f:
    json.dump(ensemble_config, f, indent=4)




[4/4] Building Final Softmax Ensemble...

Softmax Weights (temperature=1.5):
  XGBoost              0.1080
  LightGBM             0.1118
  Random Forest        0.1118
  Gradient Boosting    0.1106
  Stacking             0.1076
  Voting               0.1111
  Deep NN              0.1129
  Residual NN          0.1132
  Attention NN         0.1129


In [ ]:
# ============================================================================
# COMPREHENSIVE COMPARISON
# ============================================================================

print("\n" + "="*80)
print("FINAL MODEL COMPARISON (ALL 12 MODELS)")
print("="*80)

# Combine all metrics
tree_comparison = pd.read_csv("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/reportsf/tree_models_comparison.csv")
all_metrics = pd.concat([
    tree_comparison,
    pd.DataFrame([deep_metrics, residual_metrics, attention_metrics, ensemble_metrics])
]).sort_values('auc_roc', ascending=False).reset_index(drop=True)

print(all_metrics[['model', 'auc_roc', 'precision', 'recall', 'f1', 'brier_score']].to_string(index=False))

best = all_metrics.iloc[0]
print(f"\n🏆 Best Model: {best['model']}")
print(f"   AUC-ROC: {best['auc_roc']:.4f}")
print(f"   Brier Score: {best['brier_score']:.4f} (calibration quality)")
print(f"   Optimal Threshold: {best['optimal_threshold']:.3f}")




FINAL MODEL COMPARISON (ALL 12 MODELS)
                   model  auc_roc  precision   recall       f1  brier_score
Final Ensemble (Softmax) 0.662412   0.112731 0.997798 0.202575     0.152241
             Residual NN 0.657871   0.112243 0.999119 0.201814     0.554691
                 Deep NN 0.653227   0.112674 0.998239 0.202492     0.550997
            Attention NN 0.652949   0.112815 0.997798 0.202711     0.553513
                LightGBM 0.639182   0.170905 0.509027 0.255894     0.097252
           Random Forest 0.638226   0.124969 0.895641 0.219335     0.139682
         Voting Ensemble 0.629598   0.174322 0.424483 0.247148     0.096406
       Gradient Boosting 0.622337   0.155452 0.498459 0.236994     0.096685
                 XGBoost 0.586481   0.156515 0.381330 0.221937     0.097338
       Stacking Ensemble 0.581545   0.162822 0.372963 0.226683     0.097833

🏆 Best Model: Final Ensemble (Softmax)
   AUC-ROC: 0.6624
   Brier Score: 0.1522 (calibration quality)
   Optimal Threshold

In [ ]:
# ============================================================================
# BUSINESS IMPACT ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("BUSINESS IMPACT")
print("="*80)

# Risk stratification with optimized thresholds
high_threshold = 0.60
medium_threshold = 0.35

risk_categories = pd.cut(y_proba_ensemble,
                        bins=[0, medium_threshold, high_threshold, 1.0],
                        labels=['Low', 'Medium', 'High'])

# Calculate actual rates per tier (calibration check)
for risk in ['Low', 'Medium', 'High']:
    mask = risk_categories == risk
    count = mask.sum()
    actual_rate = y_test[mask].mean() * 100
    predicted_rate = y_proba_ensemble[mask].mean() * 100
    calibration_error = abs(predicted_rate - actual_rate)

    print(f"\n{risk} Risk:")
    print(f"  Patients: {count:,} ({count/len(y_test)*100:.1f}%)")
    print(f"  Predicted: {predicted_rate:.1f}%")
    print(f"  Actual: {actual_rate:.1f}%")
    print(f"  Calibration error: {calibration_error:.1f}%")

# Financial projections (50K annual patients)
total_patients = 50000
scale_factor = total_patients / len(y_test)

high_count = int((risk_categories == 'High').sum() * scale_factor)
medium_count = int((risk_categories == 'Medium').sum() * scale_factor)

# Intervention costs and prevention rates
high_intervention_cost = 600
medium_intervention_cost = 200
high_prevention_rate = 0.40  # Improved with better model
medium_prevention_rate = 0.25

# Calculate savings
high_prevented = high_count * high_prevention_rate
medium_prevented = medium_count * medium_prevention_rate
total_prevented = high_prevented + medium_prevented

high_savings = high_prevented * COST_FN - high_count * high_intervention_cost
medium_savings = medium_prevented * COST_FN - medium_count * medium_intervention_cost
total_savings = high_savings + medium_savings

implementation_cost = 500000
roi = (total_savings / (implementation_cost + high_count * high_intervention_cost + medium_count * medium_intervention_cost)) * 100

print(f"\n💰 Annual Financial Impact:")
print(f"   Readmissions prevented: {total_prevented:.0f}")
print(f"   Cost avoided: ${(high_prevented + medium_prevented) * COST_FN:,.0f}")
print(f"   Intervention cost: ${high_count * high_intervention_cost + medium_count * medium_intervention_cost:,.0f}")
print(f"   Net benefit: ${total_savings:,.0f}")
print(f"   ROI: {roi:.1f}%")




BUSINESS IMPACT

Low Risk:
  Patients: 11,350 (55.8%)
  Predicted: 30.6%
  Actual: 7.0%
  Calibration error: 23.6%

Medium Risk:
  Patients: 8,764 (43.1%)
  Predicted: 40.0%
  Actual: 15.6%
  Calibration error: 24.4%

High Risk:
  Patients: 240 (1.2%)
  Predicted: 69.7%
  Actual: 44.6%
  Calibration error: 25.1%

💰 Annual Financial Impact:
   Readmissions prevented: 5618
   Cost avoided: $98,308,000
   Intervention cost: $4,659,000
   Net benefit: $93,649,000
   ROI: 1815.3%


In [ ]:
# ============================================================================
# VISUALIZATIONS
# ============================================================================

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Model ranking
colors = ['#2ecc71' if i == 0 else '#3498db' for i in range(len(all_metrics))]
axes[0, 0].barh(range(len(all_metrics)), all_metrics['auc_roc'], color=colors)
axes[0, 0].set_yticks(range(len(all_metrics)))
axes[0, 0].set_yticklabels(all_metrics['model'], fontsize=9)
axes[0, 0].set_xlabel('AUC-ROC')
axes[0, 0].set_title('Model Performance Ranking', fontweight='bold')
axes[0, 0].axvline(x=0.80, color='red', linestyle='--', label='Target: 0.80')
axes[0, 0].legend()
axes[0, 0].invert_yaxis()

# Plot 2: Calibration (Brier scores)
axes[0, 1].barh(range(len(all_metrics)), all_metrics['brier_score'], color=colors)
axes[0, 1].set_yticks(range(len(all_metrics)))
axes[0, 1].set_yticklabels(all_metrics['model'], fontsize=9)
axes[0, 1].set_xlabel('Brier Score (lower = better)')
axes[0, 1].set_title('Probability Calibration Quality', fontweight='bold')
axes[0, 1].invert_yaxis()

# Plot 3: ROC curves (top 5)
for idx, row in all_metrics.head(5).iterrows():
    if row['model'] == 'Final Ensemble (Softmax)':
        proba = y_proba_ensemble
    elif row['model'] in all_preds:
        proba = all_preds[row['model']]
    else:
        continue

    fpr, tpr, _ = roc_curve(y_test, proba)
    axes[1, 0].plot(fpr, tpr, linewidth=2, label=f"{row['model'][:20]} ({row['auc_roc']:.3f})")

axes[1, 0].plot([0, 1], [0, 1], 'k--', linewidth=2)
axes[1, 0].set_xlabel('False Positive Rate')
axes[1, 0].set_ylabel('True Positive Rate')
axes[1, 0].set_title('ROC Curves - Top 5 Models', fontweight='bold')
axes[1, 0].legend(fontsize=9)

# Plot 4: Training history
axes[1, 1].plot(history_deep.history['AUC'], label='Deep NN', linewidth=2)
axes[1, 1].plot(history_residual.history['AUC'], label='Residual NN', linewidth=2)
axes[1, 1].plot(history_attention.history['AUC'], label='Attention NN', linewidth=2)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('AUC')
axes[1, 1].set_title('Neural Network Training History', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('final_comprehensive_results.png', dpi=300, bbox_inches='tight')
plt.close()



In [ ]:
# ============================================================================
# SAVE RESULTS
# ============================================================================

all_metrics.to_csv("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/reportsf/final_all_models_comparison.csv", index=False)

np.savez("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/modelsf/final_predictions.npz",
         y_test=y_test.values,
         final_ensemble=y_proba_ensemble,
         deep_nn=y_proba_deep,
         residual_nn=y_proba_residual,
         attention_nn=y_proba_attention)

final_results = {
    'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'best_model': best['model'],
    'best_auc': float(best['auc_roc']),
    'best_brier': float(best['brier_score']),
    'models_trained': len(all_metrics),
    'business_impact': {
        'annual_patients': total_patients,
        'readmissions_prevented': float(total_prevented),
        'net_benefit': float(total_savings),
        'roi_percentage': float(roi)
    }
}

with open("/content/drive/MyDrive/BerkeleyML/CapstoneHospitalReadmission/reportsf/final_results_complete.json", 'w') as f:
    json.dump(final_results, f, indent=4)

print("\n" + "="*80)
print("✓ Complete! All models trained, evaluated, and saved.")
print("="*80)
print(f"\nFiles generated:")
print("  - final_comprehensive_results.png")

print(f"\n🎯 Summary:")
print(f"  Best model: {best['model']} (AUC: {best['auc_roc']:.4f})")
print(f"  Annual value: ${total_savings:,.0f}")
print(f"  Lives improved: {int(total_prevented)}")



✓ Complete! All models trained, evaluated, and saved.

Files generated:
  - final_comprehensive_results.png

🎯 Summary:
  Best model: Final Ensemble (Softmax) (AUC: 0.6624)
  Annual value: $93,649,000
  Lives improved: 5617
